In [2]:
import pandas as pd
import matplotlib.pyplot as plt    
import numpy as np

## Data
- ID 13306385 is NAPIAS CREEK BELOW ARNETT CREEK NEAR LEESBURG, ID. This is a HEADWATER site
- ID 13039500 is HENRYS FORK NR LAKE ID, below Henrys Lake. This a site below a reservoir
- ID 13237920 is MIDDLE FORK PAYETTE RIVER NR CROUCH ID. This is a HEADWATER site
- ID 13247500 is PAYETTE RIVER NR HORSESHOE BEND ID. This is below a damn somewhere upstream, but also has other upstream free flowing tributaries. 

In [4]:
#load data 

napias_creek = pd.read_csv('../data/13306385_1980_2020.csv')
henrys_fork = pd.read_csv('../data/13039500_1980_2020.csv')
mf_payette = pd.read_csv('../data/13237920_1980_2020.csv')
payette = pd.read_csv('../data/13247500_1980_2020.csv')